In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [518 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,247 kB]
Get:10 http://a

In [2]:
from selenium import webdriver # 웹 브라우저 자동화
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time # 시간 지연
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [19]:
login_url = 'https://member.melon.com/muid/web/login/login_informM.htm'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# login
driver.get(login_url)
driver.find_element(By.NAME, 'id').send_keys('myomyo87')
driver.find_element(By.NAME, 'pwd').send_keys('exoplanet7!')
driver.find_element(By.ID, 'btnLogin').click()

In [49]:
import requests
from bs4 import BeautifulSoup

driver.get("https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=528392843")

# 페이지 소스 가져오기
page_source = driver.page_source

# Beautiful Soup로 파싱
soup = BeautifulSoup(page_source, 'html.parser')

# user_id 가져오기
user_id = soup.select_one(".user_id.ellipsis").text

# Playlist title
playlist_title = soup.select_one('div.entry > div.info > div.song_name').text.strip()

# tag 리스트 가져오기
tags = [tag.text for tag in soup.select(".tag_list.type03 .tag_item span")]

# Songs information
songs = soup.select('tbody tr')
songs_info = []

for song in songs:
    # 차트 페이지에서의 정보 수집
    title = song.select_one('.ellipsis.rank01 a').text
    rank = song.select_one('.rank').text
    singer = song.select_one('.ellipsis.rank02 a').text
    album = song.select_one('.ellipsis.rank03 a').text
    songid = song.select_one('.like')['data-song-no']

    # 상세 페이지로 이동
    BASE_DETAIL_URL = "https://www.melon.com/song/detail.htm?songId="
    detail_url = BASE_DETAIL_URL + songid
    driver.get(detail_url)
    detail_soup = bs(driver.page_source, 'html.parser')

    # 상세 페이지에서의 정보 수집
    genre = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').find_next_sibling('dd').text.strip()
    release_date = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').text.strip()
    album_image_url = detail_soup.select_one('.thumb .image_typeAll img')['src']
    likes = detail_soup.select_one('#d_like_count').text.strip()
    comments_count = detail_soup.select_one('#revCnt').text.strip()
    lyric_area = detail_soup.select_one('.lyric')
    for br in lyric_area.find_all("br"):
      br.replace_with("\n")
    lyrics = lyric_area.text.strip()

    composers = []
    lyricists = []
    arrangers = []

    # 각 li 항목을 반복 처리
    for li in detail_soup.select('.list_person.clfix li'):
      artist_name_el = li.select_one('.ellipsis .artist_name')
      artist_type_el = li.select_one('.meta .type')

      if artist_name_el and artist_type_el:  # 두 항목이 모두 존재할 때만 처리
        artist_name = artist_name_el.text.strip()
        artist_type = artist_type_el.text.strip()

        if artist_type == "작곡":
            composers.append(artist_name)
        elif artist_type == "작사":
            lyricists.append(artist_name)
        elif artist_type == "편곡":
            arrangers.append(artist_name)
    print("Composers:", composers)
    print("Lyricists:", lyricists)
    print("Arrangers:", arrangers)
    print("lyric : ", lyrics)

    songs_info.append([user_id, playlist_title, tags, songid, title, rank, singer, album, likes, release_date, genre, comments_count, lyrics, album_image_url, ", ".join(composers), ", ".join(lyricists), ", ".join(arrangers)])
    print(songs_info)


Composers: ['네이슨 (NATHAN)', '스텔라장 (Stella Jang)']
Lyricists: ['스텔라장 (Stella Jang)', '올티 (Olltii)']
Arrangers: ['네이슨 (NATHAN)']
lyric :  No money no car no rolex
하지만 너 만으로도 난 만족해
I got you and you got me
and we got us
I got you and you got me
and we got us
No money no car no rolex
하지만 너 만으로도 난 만족해
I got you and you got me
and we got us
I got you and you got me
and we got us
솔직히 돈 없는 거 조금은 불편해
시계는 몰라도 차는 있었으면 해
걱정은 하지 마 내가 다 사줄게
I’ll get ya everything ya want
cuz Imma be your 호구 man
나를 다 빨아 드셔
배부를 때까지 마셔
힘들면 잠깐 쉬어도 되니까
편하게 생각하셔
아직은 없어서 못해 but one day
I’ll be on the top
of the money swag
Then I’ll buy you everything
that you want
그러니까 누나만 믿어
언젠가 밤이 너무 짙어
우리 가던 길을 잃어
헤매도 같이 발을 딛어
남의 개소린 다 씹어
그게 롱런의 비결
누가 뭐래도 기억해
난 너의 No 1 Cheerleader
No money no car no rolex
하지만 너 만으로도 난 만족해
I got you and you got me
and we got us
I got you and you got me
and we got us
No money no car no rolex
하지만 너 만으로도 난 만족해
I got you and you got me
and we got us
I got you and you got me
and we got us
돈은 전부는 아니지만
대부분인 건 맞

In [50]:
# CSV 파일로 저장
import csv
with open('playlist_10.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["UserID", "PlaylistTitle", "Tags", "SongID", "Title", "Rank", "Singer", "Album", "Likes", "ReleaseDate", "Genre", "CommentsCount", "Lyrics", "AlbumImageURL", "Composers", "Lyricists", "Arrangers"])
    writer.writerows(songs_info)